# Evaluation Results of Adverse Drug Reactions (ADRs) in SIDER

Import necessary modules: 

In [1]:

import numpy as np
import pandas as pd
from ADRprofilePrediction import Pairs2Mat, evaluation
from Models import loadHyperpar
import json

In [2]:
import sklearn
print(sklearn.__version__)

0.24.2


## Load data

Load the feature data in to a dictionary. Drug-target, drug-enzyme, drug-chemical structure fingerprint, drug-gene interaction, drug-transporter, drug-pathway and drug-indication are included.

In [3]:
features_dict = {
    "target":Pairs2Mat(path="data/drug_target.tsv",colname1="0",colname2="1"),
    "enzyme":Pairs2Mat(path="data/drug_enzyme.tsv",colname1="0",colname2="1"),
    "Chem":pd.read_csv("data/drug_chemsfp.tsv",sep = "\t",header=0,index_col=0),
    "DGI":Pairs2Mat(path="data/interactions.tsv",colname1="drug_claim_name",colname2="gene_name"),
    "transporter":Pairs2Mat(path="data/drug_transporter.tsv",colname1="0",colname2="1"),
    "pathway":Pairs2Mat(path="data/drug_pathway.tsv",colname1="0",colname2="1"),
    "indication":Pairs2Mat(path="data/drug_indication.tsv",colname1="1_x",colname2="6")
}


Load ADR data from SIDER and OFFSIDES. Variable SEs is a dict that stores ADR data. Variable filter controls the frequency of the ADR. When filter is "all", only the ADRs with extremely low frequencies are removed; when filter is "rare" only frequency less than 50 were used.


In [4]:
filter = "all"
SEs = {}
if filter == "all":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums >= 5)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums >= 5]
elif filter == "rare":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums < 50)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums < 50]


## Set variables

The variables below includes all the options for the code.

- features_names: This varible is the list of all the features including the target feature, the enzyme feature, chemical structure fingerprint (Chem), drug-gene interaction (DGI), the transporter featrue, the pathway feature, the indication feature.
- SE_names: ADR data from SIDER.
- methods: This option is machine learning methods used for prediction. Only the top three methods were compared, SKR, VKR and KRR.
- metrice_names: Metric we used to evaluate the performance of methods: AUPR, AUROC, AUPR per drug, AUROC per drug, AUPR+AUROC and AUPR+AUROC per drug.
- SE_name: The used ADR data in this file.
- metric: We used AUPR as the tuning metrice in Nested CV and CV.

In [7]:
features_names = ["target", "enzyme", "Chem", "DGI", "transporter", "pathway", "indication"]
# SEs_names = ["SIDER", "OFFSIDES"]
methods = ["SKR", "KRR", "VKR", "Naive", "LNSM_RLN", "LNSM_jaccard"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR", "SVM", "OCCA", "SCCA", "RF", "BRF"]
# tuning_metrices=["AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
metrice_names = ["AUPR+AUROC", "AUPR+AUROCperdrug", "AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
SEs_name = "SIDER"
metrice = "AUPR"

Set the variables for hyperparameters. We summarized 3 types of hyperparameters (SVM, RF and BRF are not competitive and time-consuming, and were tuned and trained in a seperated file -- SVM_RF.ipynb): 
 - A: This hyperparameters are tuned according to the step $\dots, 10^{-1}, 10^{0}, 10^{1}, \dots$ ($\sigma_X$ and $\sigma_Y$ does not change during tuning so they can be set as $10$ and $100$ respectively).
 - B: This hyperparameters are in $[0,1]$ and tuned according to the step $0, 0.1, \dots, 1$.
 - C: This hyperparameters are tuned based on $5, 10, 15, \dots$.

In [8]:
A = 10**np.arange(-2, 3, 1, dtype=float)
B = np.arange(0.1, 1, 0.1, dtype=float)
C = np.arange(5, 20, 5, dtype=int)
A10 = 10**np.arange(1, 2, 1, dtype=float)
A100 = 10**np.arange(2, 3, 1, dtype=float)
all_hyperparlist = {
    "SKR":[A,B,A10,A100], 
    "KRR":[A,A],
    "VKR":[A,A,C], 
    "Naive":[], 
    "LNSM_RLN":[B,A], 
    "LNSM_jaccard":[B], 
    # "SVM":[A,A,A], 
    # "RF":[C], 
    # "BRF":[C]
}

Set dictionaries to store the tuned hyperparameters and the results of CV and Nested CV.

In [10]:
hyperparsOut = {}
hyperparsOut["nested_cv"] = {}
hyperparsOut["cv"] = {}
results = {}
results["nested_cv"] = {}
results["cv"] = {}

## Nested CV and CV

Load tuned hyperparameters. If fully rerunning the tuning step of Nested CV and CV is required, please skip loading variable hyperpars and remove the option `hyperparfixed` of the function `evaluation()`.

In [55]:
# Open and read the JSON file
with open(f'results/hyperpars_{SEs_name}.xml', 'r') as xml_file:
    hyperpars = json.load(xml_file)

In [17]:
for method in methods:
    validation = "nested_cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        # Load the hyperparameter combination
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs[SEs_name], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

    validation = "cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs[SEs_name], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

using feature target
The SKR requires hyperparameter lambda, c, sigma_X, sigma_Y
---------- nested cv start ----------
Fold: 0
number of hyperpars combination:  45
first few training idx:  [ 57  75 258 281 294 362 474 479 680 698]
first few testing idx:  [ 70 151 209 236 411 438 439 625 657 704]
--- tuning end ---
target size: 142
------ best hyper pars:  (0.01, 0.4, 10, 100) ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.46165763532831294
AUROCperdrug: 0.8978972570529985
AUPR+AUROCperdrug: 1.3595548923813114
AUPR: 0.41537133982867747
AUROC: 0.8687649857050729
AUPR+AUROC: 1.2841363255337503
-----------
Fold: 1
number of hyperpars combination:  45
first few training idx:  [ 70 151 209 236 411 438 439 625 657 704]
first few testing idx:  [ 57  75 258 281 294 362 474 479 680 698]
--- tuning end ---
target size: 142
------ best hyper pars:  (0.01, 0.4, 10, 100) ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.46629645108222045
AUROCperdrug: 0.9021824148263183
AUPR+AUROCperd

/home/yezhao/anaconda3/envs/tf-2/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


VKR ends:
-----------
AUPRperdrug: 0.4414199494233323
AUROCperdrug: 0.8888652896768148
AUPR+AUROCperdrug: 1.3302852391001472
AUPR: 0.3951850082752013
AUROC: 0.8752621672235312
AUPR+AUROC: 1.2704471754987325
-----------
Fold: 3
number of hyperpars combination:  75
first few training idx:  [ 70 151 209 236 411 438 439 625 657 704]
first few testing idx:  [ 14  21  36  93 153 340 352 403 418 538]
--- tuning end ---
target size: 142
------ best hyper pars:  (10, 0.01, 15) ------
VKR starts:
VKR ends:
-----------
AUPRperdrug: 0.47833690246522237
AUROCperdrug: 0.8972055841573693
AUPR+AUROCperdrug: 1.3755424866225916
AUPR: 0.4275471452853586
AUROC: 0.8737962850631014
AUPR+AUROC: 1.30134343034846
-----------
Fold: 4
number of hyperpars combination:  75
first few training idx:  [ 70 151 209 236 411 438 439 625 657 704]
first few testing idx:  [ 15  50 158 198 208 278 541 564 579 686]
--- tuning end ---
target size: 142
------ best hyper pars:  (10, 0.01, 15) ------
VKR starts:
VKR ends:
-------

/home/yezhao/anaconda3/envs/tf-2/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


VKR ends:
-----------
AUPRperdrug: 0.4503620748771144
AUROCperdrug: 0.8859509594708835
AUPR+AUROCperdrug: 1.3363130343479979
AUPR: 0.41586098158259405
AUROC: 0.8756739195173937
AUPR+AUROC: 1.2915349010999877
-----------
Fold: 4
number of hyperpars combination:  75
first few training idx:  [123 155 179 212 231 458 481 485 510 523]
first few testing idx:  [ 49  83 163 209 214 253 278 387 466 479]
--- tuning end ---
target size: 108
------ best hyper pars:  (10, 1, 15) ------
VKR starts:
VKR ends:
-----------
AUPRperdrug: 0.43902551850450117
AUROCperdrug: 0.8828594219542886
AUPR+AUROCperdrug: 1.3218849404587898
AUPR: 0.38343629758930986
AUROC: 0.8539107291954282
AUPR+AUROC: 1.2373470267847382
-----------
Mean AUPRperdrug: 0.4438169187289612, std: 0.012550276496160731
Mean AUROCperdrug: 0.8851815177814834, std: 0.0020786082582573556
Mean AUPR+AUROCperdrug: 1.3289984365104446, std: 0.014206997525150606
Mean AUPR: 0.39821476956867646, std: 0.024372974363863285
Mean AUROC: 0.8678675386677372,

/home/yezhao/anaconda3/envs/tf-2/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


VKR ends:
-----------
AUPRperdrug: 0.4516524710091313
AUROCperdrug: 0.8967192125001915
AUPR+AUROCperdrug: 1.3483716835093227
AUPR: 0.4032362624419107
AUROC: 0.874010357419414
AUPR+AUROC: 1.2772466198613248
-----------
Fold: 2
number of hyperpars combination:  75
first few training idx:  [ 94 124 142 193 241 339 379 438 651 689]
first few testing idx:  [  5 106 111 220 227 306 395 463 520 538]
--- tuning end ---
target size: 150
------ best hyper pars:  (10, 0.1, 15) ------
VKR starts:
VKR ends:
-----------
AUPRperdrug: 0.45234098226399716
AUROCperdrug: 0.8972715794191863
AUPR+AUROCperdrug: 1.3496125616831836
AUPR: 0.4361310366318718
AUROC: 0.8880710401596048
AUPR+AUROC: 1.3242020767914766
-----------
Fold: 3
number of hyperpars combination:  75
first few training idx:  [ 94 124 142 193 241 339 379 438 651 689]
first few testing idx:  [ 42 120 130 249 307 417 426 437 654 706]
--- tuning end ---
target size: 150
------ best hyper pars:  (10, 0.01, 15) ------
VKR starts:
VKR ends:
-------

## Save Output

Store the tuned hyperparameters for reproducing results.

In [24]:
# with open(f'results/hyperpars_{SEs_name}.xml', 'w') as xml_file:
#    json.dump(hyperparsOut, xml_file)

Store the results of Nested CV and CV.

In [25]:
with open(f'results/results_{SEs_name}_{filter}.xml', 'w') as xml_file:
   json.dump(results, xml_file)

## Reorganize the Results and Calculate the P-value of Method Comparison

Load the results of Nested CV and CV.

In [19]:
with open(f'results/results_{SEs_name}_{filter}.xml', 'r') as xml_file:
    results = json.load(xml_file)

Orgainize the results of Nested CV into table.

In [20]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            })
            df = pd.concat([df, temp_df], ignore_index=True)

custom_order = ["pathway","Chem", "DGI",  "indication", "target", "transporter", "enzyme"]
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], aggfunc={'score': ["mean","std"]}, columns=["metric"])
df3 = df2.sort_index(axis=1, level='metric').sort_index(level='feature')
df3.to_excel(f'results/nested_cv_results_{SEs_name}_{filter}.xlsx')
df3

score                               \
                               mean        std              mean   
metric                   AUPR+AUROC AUPR+AUROC AUPR+AUROCperdrug   
feature     method                                                 
pathway     SKR            1.334705   0.025306          1.404288   
            KRR            1.314502   0.026516          1.392919   
            VKR            1.319026   0.024290          1.375027   
            Naive          1.242813   0.016862          1.313980   
            LNSM_RLN       1.195971   0.056031          1.274742   
            LNSM_jaccard   0.826184   0.109095          0.979166   
Chem        SKR            1.275559   0.030553          1.356358   
            KRR            1.259418   0.030815          1.344612   
            VKR            1.273031   0.030400          1.348570   
            Naive          1.240348   0.025632          1.314588   
            LNSM_RLN       1.264327   0.025634          1.354326   
            LNSM_jaccard   0.545278   0.036824          0.556134   
DGI         SKR            1.291974   0.021484          1.351928   
            KRR            1.282242   0.020611          1.345147   
            VKR            1.299818   0.019064          1.344537   
            Naive          1.229446   0.010575          1.304276   
            LNSM_RLN       1.090868   0.039387          1.125397   
            LNSM_jaccard   0.573410   0.030517          0.651733   
indication  SKR            1.339038   0.012389          1.364707   
            KRR            1.334615   0.012151          1.360746   
            VKR            1.351819   0.006965          1.351034   
            Naive          1.218148   0.006685          1.295162   
            LNSM_RLN       1.161336   0.029818          1.195451   
            LNSM_jaccard   0.726010   0.057148          0.839755   
target      SKR            1.285433   0.017573          1.363857   
            KRR            1.282878   0.021430          1.362296   
            VKR            1.270189   0.018535          1.345008   
            Naive          1.232828   0.008118          1.309231   
            LNSM_RLN       1.013869   0.041392          1.017641   
            LNSM_jaccard   0.627742   0.057291          0.741293   
transporter SKR            1.288142   0.047105          1.365046   
            KRR            1.290084   0.046446          1.366099   
            VKR            1.290404   0.043731          1.366746   
            Naive          1.273007   0.043629          1.349751   
            LNSM_RLN       1.177767   0.093776          1.278029   
            LNSM_jaccard   0.409231   0.088376          0.387224   
enzyme      SKR            1.259023   0.027410          1.326630   
            KRR            1.258437   0.031869          1.324738   
            VKR            1.266082   0.034161          1.328998   
            Naive          1.251136   0.021576          1.322066   
            LNSM_RLN       1.130485   0.078116          1.195931   
            LNSM_jaccard   0.511004   0.062092          0.513626   

                                                                          \
                                       std      mean       std      mean   
metric                   AUPR+AUROCperdrug     AUROC     AUROC      AUPR   
feature     method                                                         
pathway     SKR                   0.013062  0.877330  0.007133  0.457375   
            KRR                   0.015386  0.863253  0.007199  0.451248   
            VKR                   0.013385  0.879535  0.005165  0.439491   
            Naive                 0.009992  0.859248  0.006301  0.383564   
            LNSM_RLN              0.031226  0.806585  0.025313  0.389386   
            LNSM_jaccard          0.077062  0.632716  0.062644  0.193468   
Chem        SKR                   0.023997  0.867037  0.009402  0.408522   
            KRR                   0.021969  0.854482  0.010330  0

Orgainize the results of CV into table.

In [21]:
df = pd.DataFrame()
for m, fs in results["cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            },index=["1"])
            df = pd.concat([df, temp_df], ignore_index=True)
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], columns="metric")
df2.to_excel(f'results/cv_results_{SEs_name}_{filter}.xlsx')
df2

score                                        \
metric                   AUPR+AUROC AUPR+AUROCperdrug     AUROC      AUPR   
feature     method                                                          
pathway     SKR            1.258435          1.345329  0.859145  0.399290   
            KRR            1.260420          1.345745  0.859982  0.400438   
            VKR            1.257367          1.343312  0.858144  0.399223   
            Naive          1.258257          1.343673  0.858629  0.399628   
            LNSM_RLN       1.233189          1.295248  0.828848  0.404342   
            LNSM_jaccard   0.406648          0.526215  0.332758  0.073890   
Chem        SKR            1.265030          1.353039  0.870026  0.395004   
            KRR            1.253668          1.339958  0.864452  0.389216   
            VKR            1.263963          1.344892  0.865968  0.397995   
            Naive          1.244534          1.331759  0.859970  0.384565   
            LNSM_RLN       1.279073          1.370479  0.865537  0.413536   
            LNSM_jaccard   0.584896          0.576953  0.492132  0.092763   
DGI         SKR            1.257012          1.344865  0.868380  0.388632   
            KRR            1.249470          1.332669  0.862767  0.386703   
            VKR            1.247629          1.329236  0.861516  0.386113   
            Naive          1.244041          1.330267  0.860593  0.383449   
            LNSM_RLN       1.117658          1.145836  0.762991  0.354667   
            LNSM_jaccard   0.820838          0.967887  0.641288  0.179550   
indication  SKR            1.246931          1.334537  0.874554  0.372378   
            KRR            1.214287          1.309966  0.858378  0.355909   
            VKR            1.216693          1.309215  0.859245  0.357449   
            Naive          1.217063          1.309813  0.859150  0.357913   
            LNSM_RLN       1.261872          1.243185  0.828632  0.433240   
            LNSM_jaccard   0.853548          0.872819  0.615160  0.238388   
target      SKR            1.241710          1.328310  0.860791  0.380919   
            KRR            1.242772          1.329486  0.861271  0.381501   
            VKR            1.240494          1.327335  0.859854  0.380640   
            Naive          1.241070          1.325375  0.859281  0.381789   
            LNSM_RLN       1.045623          1.076646  0.725284  0.320339   
            LNSM_jaccard   0.653125          0.769810  0.522780  0.130345   
transporter SKR            1.308501          1.390971  0.862857  0.445644   
            KRR            1.307489          1.390034  0.862605  0.444884   
            VKR            1.306653          1.389570  0.861721  0.444931   
            Naive          1.306825          1.390115  0.862142  0.444683   
            LNSM_RLN       1.211446          1.307032  0.823708  0.387738   
            LNSM_jaccard   0.457103          0.411659  0.369885  0.087218   
enzyme      SKR            1.282987          1.362420  0.861606  0.421382   
            KRR            1.280636          1.360168  0.860523  0.420113   
            VKR            1.279763          1.357971  0.859796  0.419967   
            Naive          1.277520          1.358598  0.859126  0.418393   
            LNSM_RLN       1.176423          1.302459  0.811084  0.365338   
            LNSM_jaccard   0.492192          0.509867  0.413128  0.079064   

                                                   
metric                   AUROCperdrug AUPRperdrug  
feature     method                                 
pathway     SKR              0.886235    0.459094  
            KRR              0.886422    0.459323  
            VKR              0.884432    0.458879  
            Naive            0.885073    0.458600  
            LNSM_RLN         0.833196    0.462052  
            LNSM_jaccard     0.376094    0.150122  
Chem        SKR              0.896757    0.456282  
            KRR              0.890766    0.4

Calculate the P-value for method comparison based on the result of nested CV.

In [22]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
            }, index=["1"])
            temp_df2 = pd.concat([temp_df, pd.DataFrame(scores, columns=["1"]).T], axis=1)
            df = pd.concat([df, temp_df2], ignore_index=True)
for m in metrice_names:
    for f in features_names:
        df2 = df[(df["metric"] == m) & (df["feature"] == f)]
        df3 = df2.iloc[:, np.array([0, 3, 4, 5, 6, 7])]
        df4 = df3.set_index(df3.columns[0])
        df5 = df4.T.ptests(paired=True, stars=False)
        df5.to_excel(f'results/pvalue_{SEs_name}_{filter}_{f}_{m}.xlsx')